# Intro to MLOps using ZenML

## 🌍 Overview

This repository is a minimalistic MLOps project intended as a starting point to learn how to put ML workflows in production. It features: 

- A feature engineering pipeline that loads data and prepares it for training.
- A training pipeline that loads the preprocessed dataset and trains a model.
- A batch inference pipeline that runs predictions on the trained model with new data.

Follow along this notebook to understand how you can use ZenML to productionalize your ML workflows!

<img src="assets/pipelines_overview.png" alt="Pipelines Overview">

## Run on Colab

You can use Google Colab to see ZenML in action, no signup / installation
required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/run.ipynb)

# 👶 Step 0. Install Requirements

Let's install ZenML to get started. First we'll install the latest version of
ZenML as well as the `sklearn` integration of ZenML:

In [ ]:
!pip install "zenml[server]"

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():
    # Install Cloudflare Tunnel binary
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb


In [ ]:
!zenml integration install sklearn mlflow -y

import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

Please wait for the installation to complete before running subsequent cells. At
the end of the installation, the notebook kernel will automatically restart.

Optional: If you are using ZenML Cloud, execute the following cell with your tenant URL. Otherwise ignore.

In [ ]:
zenml_server_url = "PLEASE_UPDATE_ME"  # in the form "https://URL_TO_SERVER"

!zenml connect --url $zenml_server_url

In [1]:
# Initialize ZenML and set the default stack
!zenml init

!zenml stack set default

⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/template-starter/template.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at 
/home/htahir1/workspace/zenml_io/template-starter/template.
⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/template-starter/template.
⠋ Initializing ZenML repository at 
/home/htahir1/workspace/zenml_io/template-starter/template.

The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository 
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.
Active repository stack set to: 'default'
⠋ Setting the repository active stack to 'default'...t'...


In [2]:
# Do the imports at the top
from typing_extensions import Annotated
from sklearn.datasets import load_breast_cancer

import random
import pandas as pd
from zenml import step, ExternalArtifact, pipeline, ModelVersion, get_step_context
from zenml.client import Client
from zenml.logger import get_logger
from uuid import UUID

from typing import Optional, List

from zenml import pipeline

from steps import (
    data_loader,
    data_preprocessor,
    data_splitter,
    model_evaluator,
    inference_preprocessor
)

from zenml.logger import get_logger

logger = get_logger(__name__)

# Initialize the ZenML client to fetch objects from the ZenML Server
client = Client()

## 🥇 Step 1: Load your data and execute feature engineering

We'll start off by importing our data. In this quickstart we'll be working with
[the Breast Cancer](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) dataset
which is publicly available on the UCI Machine Learning Repository. The task is a classification
problem, to predict whether a patient is diagnosed with breast cancer or not.

When you're getting started with a machine learning problem you'll want to do
something similar to this: import your data and get it in the right shape for
your training. ZenML mostly gets out of your way when you're writing your Python
code, as you'll see from the following cell.

<img src="assets/feature_engineering_pipeline.png" alt="Feature engineering pipeline">

In [6]:
@step
def data_loader_simplified(
    random_state: int, is_inference: bool = False, target: str = "target"
) -> Annotated[pd.DataFrame, "dataset"]:  # We name the dataset 
    """Dataset reader step."""
    dataset = load_breast_cancer(as_frame=True)
    inference_size = int(len(dataset.target) * 0.05)
    dataset: pd.DataFrame = dataset.frame
    inference_subset = dataset.sample(inference_size, random_state=random_state)
    if is_inference:
        dataset = inference_subset
        dataset.drop(columns=target, inplace=True)
    else:
        dataset.drop(inference_subset.index, inplace=True)
    dataset.reset_index(drop=True, inplace=True)
    logger.info(f"Dataset with {len(dataset)} records loaded!")
    return dataset


The whole function is decorated with the `@step` decorator, which
tells ZenML to track this function as a step in the pipeline. This means that
ZenML will automatically version, track, and cache the data that is produced by
this function as an `artifact`. This is a very powerful feature, as it means that you can
reproduce your data at any point in the future, even if the original data source
changes or disappears. 

Note the use of the `typing` module's `Annotated` type hint in the output of the
step. We're using this to give a name to the output of the step, which will make
it possible to access it via a keyword later on.

You'll also notice that we have included type hints for the outputs
to the function. These are not only useful for anyone reading your code, but
help ZenML process your data in a way appropriate to the specific data types.

ZenML is built in a way that allows you to experiment with your data and build
your pipelines as you work, so if you want to call this function to see how it
works, you can just call it directly. Here we take a look at the first few rows
of your training dataset.

In [7]:
df = data_loader_simplified(random_state=42)
df.head()

Dataset with 541 records loaded!


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Everything looks as we'd expect and the values are all in the right format 🥳.

We're now at the point where can bring this step (and some others) together into a single
pipeline, the top-level organising entity for code in ZenML. Creating such a pipeline is
as simple as adding a `@pipeline` decorator to a function. This specific
pipeline doesn't return a value, but that option is available to you if you need.

In [8]:
@pipeline
def feature_engineering(
    test_size: float = 0.2,
    drop_na: Optional[bool] = None,
    normalize: Optional[bool] = None,
    drop_columns: Optional[List[str]] = None,
    target: Optional[str] = "target",
):
    """Feature engineering pipeline."""
    # Link all the steps together by calling them and passing the output
    # of one step as the input of the next step.
    raw_data = data_loader(random_state=random.randint(0, 100), target=target)
    dataset_trn, dataset_tst = data_splitter(
        dataset=raw_data,
        test_size=test_size,
    )
    dataset_trn, dataset_tst, _ = data_preprocessor(
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        drop_na=drop_na,
        normalize=normalize,
        drop_columns=drop_columns,
        target=target,
    )
    
    return dataset_trn, dataset_tst

We're ready to run the pipeline now, which we can do just as with the step - by calling the
pipeline function itself:

In [9]:
feature_engineering()

Initiating a new run for the pipeline: feature_engineering.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 0.267s.
Step data_splitter has started.
Step data_splitter has finished in 0.414s.
Step data_preprocessor has started.
Step data_preprocessor has finished in 0.527s.
Run feature_engineering-2023_12_08-09_19_56_071661 has finished in 1.394s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


Let's run this again with a slightly different test size, to create another dataset:

In [10]:
feature_engineering(test_size=0.3)

Initiating a new run for the pipeline: feature_engineering.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 0.294s.
Step data_splitter has started.
Step data_splitter has finished in 0.398s.
Step data_preprocessor has started.
Step data_preprocessor has finished in 0.529s.
Run feature_engineering-2023_12_08-09_19_58_112527 has finished in 1.384s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


Notice that the data loader step was **cached**, while the rest of the pipeline was rerun. 
This is because ZenML automatically determined that nothing had changed in the data loader step, 
so it didn't need to rerun it.

At this point you might be interested to view your pipeline runs in the ZenML
Dashboard. You can spin this up by executing the next cell. This will start a
server which you can access by clicking on the link that appears in the output
of the cell.

Log into the Dashboard using default credentials (username 'default' and
password left blank). From there you can inspect the pipeline or the specific
pipeline run.


In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():
    # run ZenML through a cloudflare tunnel to get a public endpoint
    !zenml up --port 8237 & cloudflared tunnel --url http://localhost:8237
else:
    !zenml up

<img src="assets/feature_engineering_dag.png" alt="Feature engineering DAG">

We can also fetch the pipeline from the server and view the results directly in the notebook:

In [12]:
client = Client()
run = client.get_pipeline("feature_engineering").last_run
print(run.name)

feature_engineering-2023_12_08-09_19_58_112527


We can also see the data artifacts that were produced by the last step of the pipeline:

In [13]:
run.steps["data_preprocessor"].outputs

{'preprocess_pipeline': ArtifactResponse(id=UUID('31d08c04-22f0-44ae-89ad-d0530114a043'), permission_denied=False, body=ArtifactResponseBody(created=datetime.datetime(2023, 12, 8, 9, 19, 59, 419488), updated=datetime.datetime(2023, 12, 8, 9, 19, 59, 419491), user=UserResponse(id=UUID('d47f2134-eb09-41f8-9b76-4f00340c0d1b'), permission_denied=False, body=UserResponseBody(created=datetime.datetime(2023, 12, 8, 9, 18, 19, 40146), updated=datetime.datetime(2023, 12, 8, 9, 18, 19, 40148), active=True, activation_token=None, full_name='', email_opted_in=None, is_service_account=False), metadata=None, name='default'), version='2', uri='/home/htahir1/.config/zenml/local_stores/b640139f-5df3-4de5-8a01-0d16f0589628/data_preprocessor/preprocess_pipeline/ed50c2a3-71b1-4ef7-a590-8b4b717732e3', type=<ArtifactType.MODEL: 'ModelArtifact'>), metadata=None, name='preprocess_pipeline'),
 'dataset_trn': ArtifactResponse(id=UUID('5f934376-6357-4f8a-a813-b7996a379ef1'), permission_denied=False, body=Artifac

In [14]:
# Read one of the datasets. This is the one with a 0.3 test split
run.steps["data_preprocessor"].outputs["dataset_trn"].load()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
428,14.530,19.34,94.25,659.7,0.08388,0.07800,0.088170,0.02925,0.1473,0.05746,...,28.39,108.10,830.5,0.10890,0.26490,0.37790,0.09594,0.2471,0.07463,1
370,19.550,23.21,128.90,1174.0,0.10100,0.13180,0.185600,0.10210,0.1989,0.05884,...,30.44,142.00,1313.0,0.12510,0.24140,0.38290,0.18250,0.2576,0.07602,0
57,14.250,22.15,96.42,645.7,0.10490,0.20080,0.213500,0.08653,0.1949,0.07292,...,29.51,119.10,959.5,0.16400,0.62470,0.69220,0.17850,0.2844,0.11320,0
332,11.660,17.07,73.70,421.0,0.07561,0.03630,0.008306,0.01162,0.1671,0.05731,...,19.74,83.61,542.5,0.09958,0.06476,0.03046,0.04262,0.2731,0.06825,1
24,18.610,20.25,122.10,1094.0,0.09440,0.10660,0.149000,0.07731,0.1697,0.05699,...,27.26,139.90,1403.0,0.13380,0.21170,0.34460,0.14900,0.2341,0.07421,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,18.050,16.15,120.20,1006.0,0.10650,0.21460,0.168400,0.10800,0.2152,0.06673,...,18.91,150.10,1610.0,0.14780,0.56340,0.37860,0.21020,0.3751,0.11080,0
106,8.726,15.83,55.84,230.9,0.11500,0.08201,0.041320,0.01924,0.1649,0.07633,...,19.62,64.48,284.4,0.17240,0.23640,0.24560,0.10500,0.2926,0.10170,1
270,16.240,18.77,108.80,805.1,0.10660,0.18020,0.194800,0.09052,0.1876,0.06684,...,25.09,126.90,1031.0,0.13650,0.47060,0.50260,0.17320,0.2770,0.10630,0
435,13.380,30.72,86.34,557.2,0.09245,0.07426,0.028190,0.03264,0.1375,0.06016,...,41.61,96.69,705.6,0.11720,0.14210,0.07003,0.07763,0.2196,0.07675,1


We can also get the artifacts directly. Each time you create a new pipeline run, a new `artifact version` is created.

You can fetch these artifact and their versions using the `client`: 

In [15]:
dataset_trn_artifact = client.get_artifact("dataset_trn")
dataset_tst_artifact = client.get_artifact("dataset_tst")
preprocessing_pipeline_artifact = client.get_artifact("preprocess_pipeline")

f"The training set has {len(dataset_trn_artifact.versions)} records, and the test set has {len(dataset_tst_artifact.versions)} records."

AttributeError: 'ArtifactResponse' object has no attribute 'versions'

If you started with a fresh install, then you would have two versions corresponding
to the two pipelines that we ran above. Let's select the latter one to move forward:    

In [16]:
# Choose the last version
dataset_trn_artifact_version = dataset_trn_artifact.latest_version
dataset_tst_artifact_version = dataset_tst_artifact.latest_version

# Load it again to verify
dataset_trn_artifact_version.load()

AttributeError: 'ArtifactResponse' object has no attribute 'latest_version'

We'll use these artifacts from above in our next pipeline

# ⌚ Step 2: Training pipeline

Now that we have our data it makes sense to train some models to get a sense of
how difficult the task is. The Breast Cancer dataset is sufficiently large and complex 
that it's unlikely we'll be able to train a model that behaves perfectly since the problem 
is inherently complex, but we can get a sense of what a reasonable baseline looks like.

We'll start with two simple models, a SGD Classifier and a Random Forest
Classifier, both batteries-included from `sklearn`. We'll train them both on the
same data and then compare their performance.

<img src="assets/training_pipeline.png" alt="Training pipeline">

In [18]:
import pandas as pd
from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from typing_extensions import Annotated
from zenml import ArtifactConfig, step
from zenml.logger import get_logger

logger = get_logger(__name__)


@step
def model_trainer(
    dataset_trn: pd.DataFrame,
    model_type: str = "sgd",
) -> Annotated[ClassifierMixin, ArtifactConfig(name="model", is_model_artifact=True)]:
    """Configure and train a model on the training dataset."""
    target = "target"
    if model_type == "sgd":
        model = SGDClassifier()
    elif model_type == "rf":
        model = RandomForestClassifier()
    else:
        raise ValueError(f"Unknown model type {model_type}")   

    logger.info(f"Training model {model}...")

    model.fit(
        dataset_trn.drop(columns=[target]),
        dataset_trn[target],
    )
    return model


Our two training steps both return different kinds of `sklearn` classifier
models, so we use the generic `ClassifierMixin` type hint for the return type.

ZenML allows you to load any version of any dataset that is tracked by the framework
directly into a pipeline using the `ExternalArtifact` interface. This is very convenient
in this case, as we'd like to send our preprocessed dataset from the older pipeline directly
into the training pipeline.

In [19]:
@pipeline
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    model_type: str = "sgd",
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """Model training pipeline.""" 
    if train_dataset_id is None or test_dataset_id is None:
        # If we dont pass the IDs, this will run the feature engineering pipeline   
        dataset_trn, dataset_tst = feature_engineering()
    else:
        # Load the datasets from an older pipeline
        dataset_trn = ExternalArtifact(id=train_dataset_id)
        dataset_tst = ExternalArtifact(id=test_dataset_id) 

    trained_model = model_trainer(
        dataset_trn=dataset_trn,
        model_type=model_type,
    )

    model_evaluator(
        model=trained_model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )

The end goal of this quick baseline evaluation is to understand which of the two
models performs better. We'll use the `evaluator` step to compare the two
models. This step takes in the model from the trainer step, and computes its score
over the testing set.

In [20]:
# Use a random forest model with the chosen datasets.
# We need to pass the ID's of the datasets into the function
training(
    model_type="rf",
    train_dataset_id=dataset_trn_artifact.id, # dataset_trn_artifact_version.id,
    test_dataset_id=dataset_tst_artifact.id # dataset_tst_artifact_version.id
)

rf_run = client.get_pipeline("training").last_run

Initiating a new run for the pipeline: training.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step model_trainer has started.
Training model RandomForestClassifier()...
Training model RandomForestClassifier()...
Training model RandomForestClassifier()...
Step model_trainer has finished in 0.310s.
Step model_evaluator has started.
Train accuracy=100.00%
Test accuracy=95.71%
Step model_evaluator has finished in 0.140s.
Run training-2023_12_08-09_21_18_513475 has finished in 0.623s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [21]:
# Use a SGD classifier
training(model_type="sgd", train_dataset_id=dataset_trn_artifact.id, test_dataset_id=dataset_tst_artifact.id)

sgd_run = client.get_pipeline("training").last_run

Initiating a new run for the pipeline: training.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step model_trainer has started.
Training model SGDClassifier()...
Training model SGDClassifier()...
Training model SGDClassifier()...
Step model_trainer has finished in 0.152s.
Step model_evaluator has started.
Train accuracy=57.94%
Test accuracy=53.37%
Step model_evaluator has finished in 0.207s.
Run training-2023_12_08-09_21_20_671807 has finished in 0.519s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


You can see from the logs already how our model training went: the
`RandomForestClassifier` performed considerably better than the `SGDClassifier`.
We can use the ZenML `Client` to verify this:

In [22]:
# The evaluator returns a float value with the accuracy
rf_run.steps["model_evaluator"].output.load() > sgd_run.steps["model_evaluator"].output.load()

True

# 💯 Step 3: Associating a model with your pipeline

You can see it is relatively easy to train ML models using ZenML pipelines. But it can be somewhat clunky to track
all the models produced as you develop your experiments and use-cases. Luckily, ZenML offers a *Model Control Plane*,
which is a central register of all your ML models.

You can easily create a ZenML `Model` and associate it with your pipelines using the `ModelVersion` object:

In [23]:
pipeline_settings = {}

# Lets add some metadata to the model to make it identifiable
pipeline_settings["model_version"] = ModelVersion(
    name="breast_cancer_classifier",
    license="Apache 2.0",
    description="A breast cancer classifier",
    tags=["breast_cancer", "classifier"],
)

In [24]:
# Let's train the SGD model and set the version name to "sgd"
pipeline_settings["model_version"].version = "sgd"

# the `with_options` method allows us to pass in pipeline settings
#  and returns a configured pipeline
training_configured = training.with_options(**pipeline_settings)

# We can now run this as usual
training_configured(model_type="sgd", train_dataset_id=dataset_trn_artifact.id, test_dataset_id=dataset_tst_artifact.id)

Initiating a new run for the pipeline: training.
Reusing registered version: (version: 2).
New model breast_cancer_classifier was created implicitly.
New model version sgd was created.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of model_trainer.
Step model_trainer has started.
Using cached version of model_evaluator.
Linking artifact output to model None version None implicitly.
Step model_evaluator has started.
Run training-2023_12_08-09_21_36_798853 has finished in 0.252s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [25]:
# Let's train the RF model and set the version name to "rf"
pipeline_settings["model_version"].version = "rf"

# the `with_options` method allows us to pass in pipeline settings
#  and returns a configured pipeline
training_configured = training.with_options(**pipeline_settings)

# Let's run it again to make sure we have two versions
training_configured(model_type="rf", train_dataset_id=dataset_trn_artifact.id, test_dataset_id=dataset_tst_artifact.id)

Initiating a new run for the pipeline: training.
Reusing registered version: (version: 1).
New model version rf was created.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of model_trainer.
Step model_trainer has started.
Using cached version of model_evaluator.
Linking artifact output to model None version None implicitly.
Step model_evaluator has started.
Run training-2023_12_08-09_21_38_756106 has finished in 0.238s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


This time, running both pipelines has created two associated **model versions**.
You can list your ZenML model and their versions as follows:

In [26]:
zenml_model = client.get_model("breast_cancer_classifier")
print(zenml_model)

print(f"Model {zenml_model.name} has {len(zenml_model.versions)} versions")

zenml_model.versions[0].version, zenml_model.versions[1].version

name='breast_cancer_classifier' license='Apache 2.0' description='A breast cancer classifier' audience=None use_cases=None limitations=None trade_offs=None ethics=None id=UUID('22a5cae2-b9f8-4289-9773-74b848436c93') created=datetime.datetime(2023, 12, 8, 9, 21, 36, 672757) updated=datetime.datetime(2023, 12, 8, 9, 21, 36, 672759) missing_permissions=False user=UserResponse(id=UUID('d47f2134-eb09-41f8-9b76-4f00340c0d1b'), permission_denied=False, body=UserResponseBody(created=datetime.datetime(2023, 12, 8, 9, 18, 19, 40146), updated=datetime.datetime(2023, 12, 8, 9, 18, 19, 40148), active=True, activation_token=None, full_name='', email_opted_in=None, is_service_account=False), metadata=None, name='default') workspace=WorkspaceResponse(id=UUID('791ceb8f-8439-47cd-a76d-c61746290ce9'), permission_denied=False, body=WorkspaceResponseBody(created=datetime.datetime(2023, 12, 8, 9, 18, 18, 797852), updated=datetime.datetime(2023, 12, 8, 9, 18, 18, 797854)), metadata=None, name='default') tags

('sgd', 'rf')

The interesting part is that ZenML went ahead and linked all artifacts produced by the
pipelines to that model version, including the two pickle files that represent our
SGD and RandomForest classifier. We can see all artifacts directly from the model
version object:

In [27]:
# Let's load the RF version
rf_zenml_model_version = client.get_model_version("breast_cancer_classifier", "rf")

# We can now load our classifier directly as well
random_forest_classifier = rf_zenml_model_version.get_artifact("model").load()

random_forest_classifier

RandomForestClassifier()

If you are a ZenML Cloud user, you can see all of this visualized in the dashboard:

<img src="assets/cloud_mcp.png" alt="Model Control Plane">

There is a lot more you can do with ZenML models, including the ability to
track metrics by adding metadata to it, or having them persist in a model
registry. However, these topics can be explored more in the
[ZenML docs](https://docs.zenml.io).

For now, we will use the ZenML model control plane to promote our best
model to `production`. You can do this by simply setting the `stage` of
your chosen model version to the `production` tag.

In [28]:
# Set our best classifier to production
rf_zenml_model_version.set_stage("production", force=True)

Of course, normally one would only promote the model by comparing to all other model
versions and doing some other tests. But that's a bit more advanced use-case. See the
[e2e_batch example](https://github.com/zenml-io/zenml/tree/main/examples/e2e) to get
more insight into that sort of flow!

Once the model is promoted, we can now consume the right model version in our
batch inference pipeline directly. Let's see how that works.

# 🫅 Step 4: Consuming the model in production

The batch inference pipeline simply takes the model marked as `production` and runs inference on it
with `live data`. The critical step here is the `inference_predict` step, where we load the model in memory
and generate predictions:

<img src="assets/inference_pipeline.png" alt="Inference pipeline">


In [29]:
@step
def inference_predict(dataset_inf: pd.DataFrame) -> Annotated[pd.Series, "predictions"]:
    """Predictions step"""
    # Get the model_version
    model_version = get_step_context().model_version

    # run prediction from memory
    predictor = model_version.load_artifact("model")
    predictions = predictor.predict(dataset_inf)

    predictions = pd.Series(predictions, name="predicted")
    ### YOUR CODE ENDS HERE ###

    return predictions


Apart from the loading the model, we must also load the preprocessing pipeline that we ran in feature engineering,
so that we can do the exact steps that we did on training time, in inference time. Let's bring it all together:

In [33]:
@pipeline
def inference(preprocess_pipeline_id: UUID):
    """Model batch inference pipeline"""
    random_state = client.get_artifact("dataset").run_metadata["random_state"].value
    target = client.get_artifact("dataset_trn").run_metadata['target'].value
    df_inference = data_loader(
        random_state=random_state, is_inference=True
    )
    df_inference = inference_preprocessor(
        dataset_inf=df_inference,
        # We use the preprocess pipeline from the feature engineering pipeline
        preprocess_pipeline=ExternalArtifact(id=preprocess_pipeline_id),
        target=target,
    )
    inference_predict(
        dataset_inf=df_inference,
    )


The way to load the right model is to pass in the `production` stage into the `ModelVersion` config this time.
This will ensure to always load the production model, decoupled from all other pipelines:

In [36]:
pipeline_settings = {}

# Lets add some metadata to the model to make it identifiable
pipeline_settings["model_version"] = ModelVersion(
    name="breast_cancer_classifier",
    version="production", # We can pass in the stage name here!
    license="Apache 2.0",
    description="A breast cancer classifier",
    tags=["breast_cancer", "classifier"],
)

version production matches one of the possible ModelStages and will be fetched using stage.


In [37]:
# the `with_options` method allows us to pass in pipeline settings
#  and returns a configured pipeline
inference_configured = inference.with_options(**pipeline_settings)

# Let's run it again to make sure we have two versions
# We need to pass in the ID of the preprocessing done in the feature engineering pipeline
# in order to avoid training-serving skew
inference_configured(
    preprocess_pipeline_id=preprocessing_pipeline_artifact.id
)

Initiating a new run for the pipeline: inference.


Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Using cached version of data_loader.
Step data_loader has started.
Using cached version of inference_preprocessor.
Step inference_preprocessor has started.
Using cached version of inference_predict.
Step inference_predict has started.
Run inference-2023_12_08-09_24_59_254310 has finished in 0.265s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


ZenML automatically links all artifacts to the `production` model version as well, including the predictions
that were returned in the pipeline. This completes the MLOps loop of training to inference:

In [40]:
# Fetch production model
production_model_version = client.get_model_version("breast_cancer_classifier", "production")

# Get the predictions artifact
production_model_version.get_artifact("predictions").load()

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    1
11    1
12    1
13    0
14    0
15    0
16    1
17    1
18    0
19    1
20    0
21    1
22    1
23    1
24    1
25    0
26    0
27    1
Name: series, dtype: int64

## Congratulations!

You're a legit MLOps engineer now! You trained two models, evaluated them against
a test set, registered the best one with the ZenML model control plane,
and served some predictions. You also learned how to iterate on your models and
data by using some of the ZenML utility abstractions. You saw how to view your
artifacts and stacks via the client as well as the ZenML Dashboard.

## Further exploration

This was just the tip of the iceberg of what ZenML can do; check out the [**docs**](https://docs.zenml.io/) to learn more
about the capabilities of ZenML. For example, you might want to:

- Run the same pipeline on a cloud stack in production.
- Track your metrics in an experiment tracker like [MLflow]().
- Learn how to transition your code from this notebook setting to a production setting.

## What next?

* If you have questions or feedback... join our [**Slack Community**](https://zenml.io/slack) and become part of the ZenML family!
* If you want to quickly get started with ZenML, check out the [ZenML Cloud](https://zenml.io/cloud).